# Overview

This notebook is used to create and solve larger scenarios for optimization. We create randomized scenarios by specifying the number of machines, days, maintenance crews available, and the strategy (1 or 2) for the optimization solve. Once the notebook is run, the new scenarios are available in the DO model `PredictiveMaintenance` for viewing and comparison.

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM.** <br>
**© Copyright IBM Corp. 2019, 2020. All Rights Reserved.<br> 
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**<br>

## Step 0: If you are running on Cloud Pak for Data as a Service (Public Cloud)

A pre-requisite of running the rest of the cells is to have:
* From the "More" menu above, select the "Insert project token" option
* Enter your "API Key" and "location" (e.g. 'us-south') in the cell below

In [ ]:
# If running on Cloud, enter your API key and instance location below.
api_key = ''
location = 'us-south'

## Step 1: Define deployed ML model endpoint and token

A pre-requisite of running this step is to have:
* Created the Decision Optimization model PredictiveMaintenance. If you have used a different name for your DO model, update the DO_model variable below
* Run the AutoAI model and deployed it. Set the ML_Deployment_ID variable below to the Deployment ID for the deployed model.

In [ ]:
DO_model="PredictiveMaintenance"
ML_Deployment_ID = ""

In [ ]:
import os, pandas as pd, requests, urllib3, json, math, numpy as np
from pprint import pprint
from ibm_watson_machine_learning import APIClient
import os

if api_key != '':
    wml_credentials = {
        "apikey": api_key,
        "url": 'https://' + location + '.ml.cloud.ibm.com'
    }
else:
    token = os.environ['USER_ACCESS_TOKEN']

    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5"
     }

client = APIClient(wml_credentials)

### Find Deployment Space by name

In [ ]:
# Specify a name for the target deployment space
SPACE_NAME = 'IntelligentMaintenance'

# Check is the space already exists
space_uid = ''
for space in client.spaces.get_details()['resources']:
    if space['entity']['name'] == SPACE_NAME:
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        break

if space_uid == '':
    print("Deployment space",SPACE_NAME,"not found.")

## Step 2: Define all basic inputs

Here we read in the days info, define all randomization parameters, and finally generate machine data that will include machine ID, production capacity, expected life, cost of maintenance, production loss percentage due to maintenance, cost of repair, production loss percentage due to repair, opportunity cost of maintence scheduled earlier than necessary, and the value of each production unit. For more detailed description of these data fields, please refer to the README project file.

In [ ]:
import random

# When running on Cloud click the option "Insert project token" - if not running on cloud project is setup below
try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project()  

df_day = pd.read_csv(project.get_file('day.csv'))
n_days = len(df_day)

n_machines = 30
min_life = 5
max_life = n_days
min_capacity = 80
max_capacity = 150

data_machine = []
cost_of_maintenance = 50
maintenance_loss = 50
cost_of_repair = 100
repair_loss = 100
loss_per_life_day = 20
production_value_unit = 10

for i in range(n_machines):
    capacity = 10*(random.randint(min_capacity, max_capacity)/10)
    life = random.randint(min_life, max_life)
    data_machine.append(("M-%02d" % (i+1), capacity, life, cost_of_maintenance, maintenance_loss, cost_of_repair, repair_loss, loss_per_life_day, production_value_unit))

df_machine = pd.DataFrame(data=data_machine, columns=['id', 'capacity', 'remaining life', "cost of maintenance", "maintenance loss", "cost of repair", "repair loss", "loss per life day", "production value unit"])

## Step 3: Generate production data, obtain predictions from deployed ML model, and generate predicted failure data

For each machine we generate production data and obtain predictions based on machine's remaining life and planned production numbers. We then use a normal distribution to generate predicted failure data, given prediction `mid` obtained from the ML model.
Note that we do not use fixed maintenance in these scenarios, so the optimization model will assign maintenance as needed in order to reach an optimal solution. 

In [ ]:
data_production = []
data_failure = []
data_fixed_maintenance = []

for machine in df_machine['id']:
    life = int(df_machine[df_machine.id==machine]['remaining life'])
    capacity = int(df_machine[df_machine.id==machine]['capacity'])
    base = random.randint(int(0.7*capacity), capacity)    
    x = [life]
    
    # generate payload data for submitting to the deployed ML model service
    payload_data = {};
    
    payload_data['life'] = float(life)
    for day in df_day['id']:
        production = min(capacity, random.randint(int(0.8*base),int(1.2*base)))
        data_production.append((machine, day, production))
        data_fixed_maintenance.append((machine, day, 0))
        x.append(production)
        payload_data[day] = float(production)

    scoring_payload = {
        "input_data":  [{
            'fields': list(payload_data.keys()),
            'values': [list(payload_data.values())]
        }]
    }
    
    response_scoring = client.deployments.score(ML_Deployment_ID, scoring_payload)
    prediction = response_scoring['predictions'][0]['values'][0][0]
    mid = int(math.floor(float(prediction)))

    print (str(x) + " --> " + str(mid))
    
    spread = random.randint(2, 6)

    n = 1000
    s = np.random.normal(mid, spread/4., n)

    data = [0 for i in range(n_days)]
    for i, day in np.ndenumerate(df_day['id']):
        t = 0
        for j in range(1000):
            if round(s[j])==i[0]+1:
                t = t+1                    
        data_failure.append((machine, day, round (100.*t/n)))
        
        
df_planned_production = pd.DataFrame(data=data_production, columns=['machine', 'day', 'production'])
df_predicted_failure = pd.DataFrame(data=data_failure, columns=['machine', 'day', 'failure'])
df_fixed_maintenance = pd.DataFrame(data=data_fixed_maintenance, columns=['machine', 'day', 'maintenance'])

## Step 4: Generate new scenarios, varying parameters for crew size and optimization strategy

Generate 4 different scenarios for `crew size = {2,3}` and the two different optimization strategies. We copy `Scenario 1` as a template, set the required parameters, and attach previously generated input data.  

In [ ]:
!pip install --user dd-scenario

from dd_scenario import *

# In order to use the solve() function on Cloud, you must provide an API key when creating the client
if api_key != '':
    client = Client(pc=pc, apikey=api_key)
else:
    client = Client()
    
fw = client.get_model_builder(name=DO_model)

for n_crew in range(2,4): 
    print ("NCrew " + str(n_crew))
    for strategy in range(1,3): 
        print ("Strategy " + str(strategy))
        data_parameters = []
        data_parameters.append(('strategy', strategy))
        data_parameters.append(('maintenance crew size', n_crew))
        data_parameters.append(('fix_maintenance', 0))
        df_parameters = pd.DataFrame(data=data_parameters, columns=['id', 'value'])

        sc = fw.get_scenario(name="Scenario 1")
        # Copy from Scenario 1 to get the right model
        sc_name = "Predicted-%02d-%02d-%02d-%01d" % (n_machines, n_days, n_crew, strategy)
        copy = fw.get_scenario(name=sc_name)
        if (copy != None):
            print (" Deleting old...")
            fw.delete_container(copy)    
        copy = sc.copy(sc_name)
        print (" Creating " + sc_name)
        copy.add_table_data("day", df_day, category="input")
        copy.add_table_data("machine", df_machine, category="input")
        copy.add_table_data("predicted_failure", df_predicted_failure, category="input")
        copy.add_table_data("planned_production", df_planned_production, category="input")
        copy.add_table_data("parameters", df_parameters, category="input")
        copy.add_table_data("fixed_maintenance", df_fixed_maintenance, category="input")

        print (" Solving...")
        copy.solve()

print ("Done...")